In [1]:
# 第四节 基于Gensim的词向量实战

## 一、导入库

In [2]:
# 如果尚未安装，请运行以下命令：
# !pip install gensim jieba

import jieba
from gensim import corpora, models
from gensim.models import Word2Vec, KeyedVectors
import warnings
warnings.filterwarnings('ignore')

## 二、Gensim工作流

In [3]:
# Step 1: 准备分词后的语料 (新闻标题)
raw_headlines = [
    "央行降息，刺激股市反弹",
    "球队赢得总决赛冠军，球员表现出色"
]
tokenized_headlines = [jieba.lcut(doc) for doc in raw_headlines]
print(f"分词后语料: {tokenized_headlines}")

# Step 2: 创建词典
dictionary = corpora.Dictionary(tokenized_headlines)
print(f"词典: {dictionary.token2id}")

# Step 3: 转换为BoW向量语料库
corpus_bow = [dictionary.doc2bow(doc) for doc in tokenized_headlines]
print(f"BoW语料库: {corpus_bow}")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dalvqw\AppData\Local\Temp\jieba.cache
Loading model cost 0.381 seconds.
Prefix dict has been built successfully.


分词后语料: [['央行', '降息', '，', '刺激', '股市', '反弹'], ['球队', '赢得', '总决赛', '冠军', '，', '球员', '表现出色']]
词典: {'刺激': 0, '反弹': 1, '央行': 2, '股市': 3, '降息': 4, '，': 5, '冠军': 6, '总决赛': 7, '球员': 8, '球队': 9, '表现出色': 10, '赢得': 11}
BoW语料库: [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)], [(5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)]]


## 三、TF-IDF与关键词权重

In [4]:
# 1. 准备语料 (新闻标题，包含财经和体育两个明显主题)
headlines = [
    "央行降息，刺激股市反弹",
    "球队赢得总决赛冠军，球员表现出色",
    "国家队公布最新一期足球集训名单",
    "A股市场持续震荡，投资者需谨慎",
    "篮球巨星刷新历史得分记录",
    "理财产品收益率创下新高"
]
tokenized_headlines = [jieba.lcut(title) for title in headlines]

# 2. 创建词典和BoW语料库
dictionary = corpora.Dictionary(tokenized_headlines)
corpus_bow = [dictionary.doc2bow(doc) for doc in tokenized_headlines]

print(f"词典大小: {len(dictionary)}")
print(f"语料库包含 {len(corpus_bow)} 篇文档")

词典大小: 35
语料库包含 6 篇文档


In [12]:
# 3. 训练TF-IDF模型
tfidf_model = models.TfidfModel(corpus_bow)

# 4. 将BoW语料库转换为TF-IDF向量表示
corpus_tfidf = tfidf_model[corpus_bow]

# 辅助函数：把 (token_id, weight) 转成 (token, weight)，并按权重降序展示
def tfidf_with_words(tfidf_vec, id2word):
    pairs = [(id2word[token_id], weight) for token_id, weight in tfidf_vec]
    return sorted(pairs, key=lambda x: x[1], reverse=True)

# 打印第一篇标题的TF-IDF向量
first_tfidf = list(corpus_tfidf)[0]
print("第一篇标题的TF-IDF向量:")
print(first_tfidf)
print("第一篇标题的TF-IDF向量(带词语):")
print(tfidf_with_words(first_tfidf, dictionary))

# 5. 对新标题应用模型
new_headline = "股市大涨，牛市来了"
new_headline_bow = dictionary.doc2bow(list(jieba.cut(new_headline)))
new_headline_tfidf = tfidf_model[new_headline_bow]
print("\n新标题的TF-IDF向量:")
print(new_headline_tfidf)

第一篇标题的TF-IDF向量:
[(0, 0.44066740566370055), (1, 0.44066740566370055), (2, 0.44066740566370055), (3, 0.44066740566370055), (4, 0.44066740566370055), (5, 0.1704734229377651)]
第一篇标题的TF-IDF向量(带词语):
[('刺激', 0.44066740566370055), ('反弹', 0.44066740566370055), ('央行', 0.44066740566370055), ('股市', 0.44066740566370055), ('降息', 0.44066740566370055), ('，', 0.1704734229377651)]

新标题的TF-IDF向量:
[(3, 0.9326446771245245), (5, 0.360796211497975)]


## 四、LDA与文档主题挖掘

In [6]:
# 1. 准备语料
headlines = [
    "央行降息，刺激股市反弹",
    "球队赢得总决赛冠军，球员表现出色",
    "国家队公布最新一期足球集训名单",
    "A股市场持续震荡，投资者需谨慎",
    "篮球巨星刷新历史得分记录",
    "理财产品收益率创下新高"
]
tokenized_headlines = [jieba.lcut(title) for title in headlines]

# 2. 创建词典和BoW语料库
dictionary = corpora.Dictionary(tokenized_headlines)
corpus_bow = [dictionary.doc2bow(doc) for doc in tokenized_headlines]

# 3. 训练LDA模型 (假设需要发现2个主题)
lda_model = models.LdaModel(corpus=corpus_bow, id2word=dictionary, num_topics=2, random_state=100)

# 4. 查看模型发现的主题
print("模型发现的2个主题及其关键词:")
for topic in lda_model.print_topics():
    print(topic)


模型发现的2个主题及其关键词:
(0, '0.045*"，" + 0.040*"公布" + 0.039*"一期" + 0.039*"名单" + 0.039*"足球" + 0.039*"最新" + 0.038*"集训" + 0.038*"国家队" + 0.037*"A股" + 0.037*"市场"')
(1, '0.066*"，" + 0.039*"篮球" + 0.039*"刷新" + 0.039*"历史" + 0.039*"记录" + 0.038*"得分" + 0.038*"巨星" + 0.037*"刺激" + 0.036*"降息" + 0.036*"反弹"')


In [13]:
# 5. 推断新文档的主题分布
new_headline = "巨星詹姆斯获得常规赛MVP"
new_headline_bow = dictionary.doc2bow(jieba.lcut(new_headline))
topic_distribution = lda_model[new_headline_bow]
print(f"\n新标题 '{new_headline}' 的主题分布:")
print(topic_distribution)


新标题 '巨星詹姆斯获得常规赛MVP' 的主题分布:
[(0, 0.27247813), (1, 0.7275219)]


## 五、Word2Vec模型实战

### 5.1 模型训练与核心参数

In [66]:
# 1. 准备语料 
headlines = [
    # 财经
    "央行降息，刺激股市反弹",
    "A股市场持续震荡，投资者需谨慎",
    "理财产品收益率创下新高",
    "证监会发布新规，规范市场交易",
    "创业板指数上涨，科技股领涨大盘",
    "房价调控政策出台，房地产市场降温",
    "全球股市动荡，影响资本市场信心",
    "分析师认为，当前股市风险与机遇并存，市场情绪复杂",

    # 体育
    "球队赢得总决赛冠军，球员表现出色",
    "国家队公布最新一期足球集训名单",
    "篮球巨星刷新历史得分记录",
    "奥运会开幕，中国代表团旗手确定",
    "马拉松比赛圆满结束，选手创造佳绩",
    "电子竞技联赛吸引大量年轻观众",
    "这支球队的每位球员都表现出色",
    "球员转会市场活跃，多支球队积极引援"
]

tokenized_headlines = [list(jieba.cut(title)) for title in headlines]

# 2. 训练Word2Vec模型
model = Word2Vec(tokenized_headlines, vector_size=50, window=3, min_count=1, sg=1)

print(f"模型训练完成！")
print(f"词汇表大小: {len(model.wv)}")
print(f"词向量维度: {model.wv.vector_size}")

# 训练完成后，所有词向量都存储在 model.wv 对象中
# model.wv 是一个 KeyedVectors 实例

模型训练完成！
词汇表大小: 93
词向量维度: 50


### 5.2 使用词向量


In [67]:
# 1. 寻找最相似的词
# 在小语料上，结果可能不完美，但能体现出模型学习到了主题内的关联
similar_to_market = model.wv.most_similar('股市')
print(f"与 '股市' 最相似的词: {similar_to_market}")

# 2. 计算两个词的余弦相似度
similarity = model.wv.similarity('球队', '球员')
print(f"\n'球队' 和 '球员' 的相似度: {similarity:.4f}")

# 3. 获取一个词的向量
market_vector = model.wv['市场']
print(f"\n'市场' 的向量维度: {market_vector.shape}")

与 '股市' 最相似的词: [('名单', 0.3699544370174408), ('央行', 0.3554984927177429), ('年轻', 0.24192844331264496), ('联赛', 0.23673078417778015), ('马拉松', 0.20006775856018066), ('证监会', 0.19390541315078735), ('每位', 0.17580750584602356), ('积极', 0.16841839253902435), ('球队', 0.16585905849933624), ('理财产品', 0.16510605812072754)]

'球队' 和 '球员' 的相似度: -0.1552

'市场' 的向量维度: (50,)


### 5.3 模型的持久化


In [68]:
# 保存词向量到文件
model.wv.save("news_vectors.kv")

# 从文件加载词向量
loaded_wv = KeyedVectors.load("news_vectors.kv")

# 加载后可以执行同样的操作
print(f"\n加载后，'球队' 和 '球员' 的相似度: {loaded_wv.similarity('球队', '球员'):.4f}")


加载后，'球队' 和 '球员' 的相似度: -0.1552
